# Part VIII — Frontend Integration  
## 35. Django + SPA / Frontend Frameworks (React/Vue/Next) — Real‑Team Architecture, Auth, CORS/CSRF, Deployment

This chapter answers the questions that decide whether your Django project stays
pleasant as the frontend grows:

- Should you build **server-rendered Django templates** or a **separate SPA**?
- If SPA: cookie sessions or JWT? Where do tokens live?
- If split frontend/backend: how do **CORS** and **CSRF** work *correctly*?
- How do you deploy it without fragile hacks?
- How do you structure code and routes so teams can collaborate without breaking
  each other?

We’ll cover both industry-common setups:

1) **Monolith**: Django serves HTML and maybe small JS enhancements  
2) **Hybrid**: Django serves templates + uses HTMX or “sprinkles”  
3) **Split**: frontend (React/Vue/Next) is separate; Django exposes APIs (DRF)

Then we’ll implement two full auth patterns you’ll see in production:

- **Cookie session + CSRF** (often best for same-site web apps and internal tools)
- **JWT (access + refresh)** (common for mobile + SPAs, but higher complexity)

---

## 35.0 Learning Outcomes

By the end, you should be able to:

1. Choose an architecture: monolith vs split, and justify it.
2. Set up a Django + DRF backend that supports a SPA frontend safely.
3. Implement one of these auth strategies correctly:
   - **Cookie session auth + CSRF**
   - **Token/JWT auth** (bearer tokens, refresh tokens)
4. Configure CORS and CSRF trusted origins properly (no dangerous wildcard setups).
5. Deploy a frontend + backend cleanly (reverse proxy routing, static builds).
6. Design API routes and frontend routes that don’t conflict.
7. Know the “gotchas” and how to test them.

---

## 35.1 Architecture Options (What Real Teams Choose)

### 35.1.1 Option A — Django templates (server-rendered) + minimal JS
**Best for:**
- content sites, admin-heavy tools, internal apps
- teams that want simplicity and fast delivery
- SEO-heavy pages

**Pros:**
- simplest security model (cookies + CSRF)
- fewer moving parts (one codebase, one deploy)
- Django forms/admin shine

**Cons:**
- very complex client-side UI can feel clunky
- if you need offline-first or heavy client state, you may fight templates

### 35.1.2 Option B — Django + HTMX (hybrid)
**Best for:**
- CRUD apps that want interactivity without SPA complexity
- teams that like server-rendered HTML but want responsiveness

**Pros:**
- keeps Django strengths
- avoids building a full frontend build pipeline (or keeps it minimal)
- progressive enhancement

**Cons:**
- still not ideal for highly complex client-side experiences
- you must manage partial templates carefully

### 35.1.3 Option C — Split frontend (React/Vue/Next) + DRF API
**Best for:**
- mobile apps + web app sharing the same backend
- teams with dedicated frontend engineers
- heavy UI, complex interactions
- microservices/platform teams

**Pros:**
- clean separation of concerns
- frontend can move fast with its own tooling and release cycle
- reusable API for multiple clients

**Cons:**
- hardest security configuration (CORS/CSRF, token storage)
- more infrastructure: two builds, two deploys, version coordination
- more ways to break user sessions with misconfiguration

---

## 35.2 Decision Matrix (Practical)

Use this as a real “architecture meeting” tool.

### Choose server-rendered Django (templates/HTMX) if:
- you mainly build forms + lists + dashboards
- SEO matters and you want it easy
- you want simplest auth model
- your team is backend-heavy

### Choose split SPA + DRF if:
- you need a rich client UI that is hard in templates
- you have multiple clients (mobile + web)
- you have a frontend team and want a modern frontend stack
- you need to share UI components and state management patterns

**Industry reality:** Many companies start with Django templates/HTMX and move to
SPA for selected parts later. You don’t have to choose one forever.

---

## 35.3 Route Design: Keep Frontend Routes and API Routes Separate

A standard approach:

- API: `/api/v1/...`
- Auth API: `/api/v1/auth/...`
- HTML pages (if any): `/...`
- Frontend SPA routes: handled by the frontend router (e.g., `/app/...`)

Example:
- `/api/v1/articles/` → DRF
- `/api/v1/orgs/acme/tasks/` → DRF
- `/admin/` → Django admin
- `/app/` → SPA entrypoint (index.html), SPA router handles deeper paths

Why this matters:
- prevents “Django 404 vs SPA 404” confusion
- makes caching, security headers, and logging clearer
- keeps reverse proxy routing simple

---

## 35.4 Deployment Topologies (Industry Standard)

### 35.4.1 Topology 1 — Same domain, reverse proxy routes `/api` to Django
Example:
- `https://example.com/app/` → frontend static
- `https://example.com/api/` → Django (DRF)
- `https://example.com/admin/` → Django

This is the cleanest for **cookies + CSRF** because it’s same-site:
- fewer CORS problems
- cookies behave predictably

### 35.4.2 Topology 2 — Separate subdomains
Example:
- frontend: `https://app.example.com`
- backend: `https://api.example.com`

Now you must manage:
- CORS (browser needs permission to read responses)
- cookies across subdomains (if using cookie auth)
- CSRF trusted origins

This is common, but it’s where most misconfig happens.

### 35.4.3 Topology 3 — Entirely separate hosts
Example:
- frontend: Vercel/Netlify
- backend: Render/Fly/VPS/K8s

Also common. Same issues as subdomains, plus:
- different deploy pipelines
- separate environments and secrets

---

## 35.5 Auth Strategy #1 (Recommended for Same-Site SPA): Cookie Session + CSRF

This is the “Django-native” approach and works beautifully when:
- frontend and backend are same-site (same domain)
- you can rely on secure cookies
- you want easy logout/revocation

### 35.5.1 How it works (exactly)
- User logs in (via Django session auth).
- Browser stores `sessionid` cookie (HttpOnly).
- SPA requests include cookies automatically (with `credentials: "include"` or
  same-origin default).
- Unsafe requests (POST/PATCH/DELETE) must include CSRF token.

### 35.5.2 DRF configuration
In `config/settings.py`:

```python
REST_FRAMEWORK = {
    "DEFAULT_AUTHENTICATION_CLASSES": [
        "rest_framework.authentication.SessionAuthentication",
    ],
    "DEFAULT_PERMISSION_CLASSES": [
        "rest_framework.permissions.IsAuthenticatedOrReadOnly",
    ],
}
```

### 35.5.3 Provide an endpoint that sets CSRF cookie for SPA
SPAs often need a way to “bootstrap” CSRF token.

Create `pages/views_csrf.py`:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import ensure_csrf_cookie
from django.views.decorators.http import require_GET


@require_GET
@ensure_csrf_cookie
def csrf(request):
    # Sets csrftoken cookie if missing.
    return JsonResponse({"status": "ok"})
```

Wire it in `pages/urls.py`:

```python
from pages.views_csrf import csrf

urlpatterns += [
    path("csrf/", csrf, name="csrf"),
]
```

**Why `ensure_csrf_cookie`?**
- ensures the `csrftoken` cookie exists even if the response doesn’t render a form
- lets SPA read it and send it back in headers

### 35.5.4 SPA fetch wrapper (React/Vue/Next) — correct CSRF usage
A robust baseline:

```ts
// api.ts (TypeScript)
function getCookie(name: string): string | null {
  const parts = document.cookie.split(";").map((c) => c.trim());
  for (const part of parts) {
    if (part.startsWith(name + "=")) {
      return decodeURIComponent(part.slice(name.length + 1));
    }
  }
  return null;
}

export async function apiFetch(
  url: string,
  options: RequestInit = {}
): Promise<Response> {
  const headers = new Headers(options.headers || {});
  headers.set("Accept", "application/json");

  const method = (options.method || "GET").toUpperCase();

  // For unsafe methods, include CSRF token.
  if (["POST", "PUT", "PATCH", "DELETE"].includes(method)) {
    const token = getCookie("csrftoken");
    if (token) headers.set("X-CSRFToken", token);
    headers.set("Content-Type", "application/json");
  }

  return fetch(url, {
    ...options,
    headers,
    credentials: "include", // include cookies (required for session auth)
  });
}
```

**Bootstrapping step:**
Call `/csrf/` on app load once:

```ts
await apiFetch("/csrf/");
```

Then your app can safely POST/PATCH/DELETE.

### 35.5.5 Common mistakes (and fixes)
- Mistake: POST without CSRF → 403  
  Fix: call `/csrf/` and send `X-CSRFToken`
- Mistake: cookies not sent → always “AnonymousUser”  
  Fix: `credentials: "include"` and correct domain/samesite/secure settings
- Mistake: cross-origin SPA + cookies without `CORS_ALLOW_CREDENTIALS`  
  Fix: configure CORS + CSRF trusted origins correctly (Section 35.7)

---

## 35.6 Auth Strategy #2 (Common for Split Frontends): JWT (Access + Refresh)

JWT is widely used but easy to misuse. The goal is:
- short-lived access token (minutes)
- longer-lived refresh token (days/weeks)
- refresh rotation if you need strong security

### 35.6.1 Install SimpleJWT (typical DRF choice)
```bash
python -m pip install djangorestframework-simplejwt
python -m pip freeze > requirements.txt
```

### 35.6.2 Configure DRF authentication
In `config/settings.py`:

```python
REST_FRAMEWORK = {
    "DEFAULT_AUTHENTICATION_CLASSES": [
        "rest_framework_simplejwt.authentication.JWTAuthentication",
    ],
}
```

### 35.6.3 Add token endpoints
In `config/urls.py`:

```python
from django.urls import path
from rest_framework_simplejwt.views import (
    TokenObtainPairView,
    TokenRefreshView,
)

urlpatterns += [
    path("api/v1/auth/jwt/", TokenObtainPairView.as_view(), name="jwt_obtain"),
    path("api/v1/auth/jwt/refresh/", TokenRefreshView.as_view(), name="jwt_refresh"),
]
```

### 35.6.4 Client-side storage (critical security decision)
Options:

#### Option A: Store access token in memory (recommended for SPAs)
- safer vs XSS than localStorage (still not perfect; XSS can still call APIs while
  page is running)
- refresh token must be handled carefully (often HttpOnly cookie or secure storage)

#### Option B: localStorage (common but risky)
- vulnerable to XSS stealing tokens
- avoid for high-security apps

#### Option C: Store refresh token in HttpOnly cookie, access token in memory
- common “best of both worlds”
- but introduces CSRF considerations if refresh is cookie-based

**Industry advice:** If you don’t have strong frontend security posture (CSP, XSS
mitigations), tokens in localStorage are a risk.

### 35.6.5 Authorization header usage
Example API wrapper:

```ts
export async function apiFetchWithBearer(
  url: string,
  token: string,
  options: RequestInit = {}
): Promise<Response> {
  const headers = new Headers(options.headers || {});
  headers.set("Accept", "application/json");
  headers.set("Authorization", `Bearer ${token}`);

  if (options.body && !headers.has("Content-Type")) {
    headers.set("Content-Type", "application/json");
  }

  return fetch(url, { ...options, headers });
}
```

### 35.6.6 Refresh flow (concept)
- request fails with 401 → refresh access token → retry once
- ensure you avoid infinite loops

This is standard SPA logic, but it must be implemented carefully.

---

## 35.7 CORS + CSRF for Split Frontend/Backend (The Correct, Safe Setup)

This is where many projects go wrong.

### 35.7.1 If you use JWT (Authorization header), you typically want:
- CORS enabled for allowed frontend origins
- credentials not required (no cookies needed for auth)

Settings (using `django-cors-headers`):

```python
CORS_ALLOWED_ORIGINS = [
    "https://app.example.com",
]
CORS_ALLOW_CREDENTIALS = False
```

### 35.7.2 If you use cookie sessions across origins, you need:
- allow credentials
- set CSRF trusted origins
- set cookies with compatible SameSite policy

Settings:

```python
CORS_ALLOWED_ORIGINS = [
    "https://app.example.com",
]
CORS_ALLOW_CREDENTIALS = True

CSRF_TRUSTED_ORIGINS = [
    "https://app.example.com",
]

SESSION_COOKIE_SAMESITE = "None"
CSRF_COOKIE_SAMESITE = "None"
SESSION_COOKIE_SECURE = True
CSRF_COOKIE_SECURE = True
```

**Important warnings:**
- Do not set `CORS_ALLOW_ALL_ORIGINS=True` with `CORS_ALLOW_CREDENTIALS=True`.
- `SameSite=None` requires `Secure=True` (HTTPS).

### 35.7.3 Frontend fetch must include credentials for cookies
```js
fetch("https://api.example.com/api/v1/...", {
  credentials: "include",
});
```

---

## 35.8 Serving a SPA from Django (Monorepo / “Single Deploy” Pattern)

A common approach for small/medium teams:
- build React/Vue with Vite
- output into Django’s static files
- Django serves `index.html` and static assets
- DRF serves `/api/v1/`

### 35.8.1 Minimal Vite build integration (conceptual)
Frontend build outputs:
- `dist/index.html`
- `dist/assets/...`

You can:
- copy `dist/assets` into `static/`
- serve `index.html` from Django templates (or as a static file)

**Simpler approach:**
- Put `index.html` into `templates/spa/index.html`
- Add a view that returns it for `/app/` routes

Example `pages/views_spa.py`:

```python
from django.views.generic import TemplateView


class SpaIndexView(TemplateView):
    template_name = "spa/index.html"
```

In `pages/urls.py`:

```python
from pages.views_spa import SpaIndexView

urlpatterns += [
    path("app/", SpaIndexView.as_view(), name="spa"),
]
```

For deep SPA routes (`/app/tasks/123`), you typically need a “catch-all” route that
still serves the SPA index. Be careful not to swallow API/admin routes.

A safe pattern:
- only catch under `/app/`

```python
from django.urls import re_path

urlpatterns += [
    re_path(r"^app/.*$", SpaIndexView.as_view(), name="spa_catchall"),
]
```

### 35.8.2 Static config you must have
- `STATIC_URL`, `STATICFILES_DIRS`, `STATIC_ROOT`
- `collectstatic` during deploy

---

## 35.9 Reverse Proxy Routing (Clean Production Pattern)

Even if frontend is separate, a reverse proxy often unifies domains.

Example Nginx “single domain” routing:

```nginx
server {
  listen 443 ssl;
  server_name example.com;

  # Frontend static (built files)
  location /app/ {
    root /var/www/frontend;
    try_files $uri /app/index.html;
  }

  # Django API
  location /api/ {
    proxy_pass http://django_upstream;
    proxy_set_header Host $host;
    proxy_set_header X-Forwarded-Proto $scheme;
    proxy_set_header X-Request-Id $request_id;
  }

  # Django admin
  location /admin/ {
    proxy_pass http://django_upstream;
    proxy_set_header Host $host;
    proxy_set_header X-Forwarded-Proto $scheme;
    proxy_set_header X-Request-Id $request_id;
  }
}
```

Why this is nice:
- browser sees one origin (`example.com`) → simplest cookie/CSRF story
- API and frontend can still be deployed independently behind the proxy

---

## 35.10 Frontend ↔ Backend Contract Discipline (Avoid “silent breaks”)

In real teams:
- backend changes can break frontend
- frontend expects certain shapes and error formats

Professional practices:
1. OpenAPI schema (you set up in API chapters)
2. Versioning (`/api/v1/`)
3. Contract tests or snapshot tests (optional)
4. Strong CI: backend tests + frontend tests
5. Deprecation policy (“fields removed only in v2”)

---

## 35.11 Common “Gotchas” (with direct fixes)

### Gotcha A: 401 vs 403 confusion in SPA
- 401: not authenticated → show login
- 403: authenticated but forbidden → show “not allowed”

Make sure your frontend distinguishes them.

### Gotcha B: CORS errors look like “network error”
Browser blocks JS from reading response; you may see no useful error body.

Fix:
- inspect browser devtools network tab
- verify CORS headers from backend
- verify preflight OPTIONS response

### Gotcha C: Cookies not set/ sent in cross-site contexts
Fix:
- correct `SameSite` and `Secure`
- use HTTPS
- set `CORS_ALLOW_CREDENTIALS=True` if cross-origin cookie auth
- ensure frontend uses `credentials: "include"`

### Gotcha D: CSRF fails only in production
Usually caused by:
- missing `CSRF_TRUSTED_ORIGINS`
- HTTPS/proxy scheme mismatch
- incorrect domain/host

Fix:
- set `CSRF_TRUSTED_ORIGINS` to real origins
- set `SECURE_PROXY_SSL_HEADER` correctly when behind proxy

---

## 35.12 Testing Strategy for SPA + Django (Practical)

### Backend
- DRF tests for:
  - auth required
  - permission denied
  - tenant scoping
  - throttling
- CORS tests (if enabled)
- CSRF tests (if using session auth)

### Frontend
- unit tests for API client wrapper (token refresh logic, error handling)
- E2E tests (Playwright) for critical flows:
  - login → list tasks → update status → export

### Integration
- staging environment tests are extremely valuable because CORS/CSRF problems often
  only appear with real domains/HTTPS.

---

## 35.13 Hands‑On Mini‑Lab (Choose One Track)

### Track A (recommended): Same-site SPA using session + CSRF
1. Keep Django and SPA under the same domain (or same dev origin).
2. Add `/csrf/` endpoint (`ensure_csrf_cookie`).
3. Use `SessionAuthentication` in DRF.
4. Build a SPA page that:
   - GETs `/api/v1/orgs/acme/tasks/`
   - PATCHes task status using CSRF header

### Track B: Split frontend/backend with JWT
1. Enable SimpleJWT endpoints.
2. Configure CORS for your SPA origin (no credentials).
3. In SPA:
   - obtain access/refresh
   - call API with Authorization header
   - implement refresh on 401 once

---

## 35.14 Exercises (Do These Before Proceeding)

1. Write a short “architecture decision record” (ADR):
   - monolith vs split
   - auth strategy
   - CORS/CSRF decisions
   - deployment topology

2. Implement one auth strategy fully and add tests:
   - session+csrf: CSRF rejection test + authenticated write test
   - JWT: token obtain test + protected endpoint returns 401 without token

3. Add a “whoami” endpoint:
   - `GET /api/v1/me/` returns `{id, username, is_staff}`
   - used by SPA to know login state

4. Document local dev workflow in README:
   - how to run backend
   - how to run frontend dev server
   - how proxying works (if you use it)

---

## 35.15 Chapter Summary

- Pick architecture based on product needs and team skills; simplest wins unless
  complexity is required.
- Same-site SPA + cookie sessions + CSRF is often the most secure and simplest for
  web apps.
- JWT is powerful for mobile/split deployments but requires careful token storage
  and rotation strategy.
- CORS is about allowing browser JS to read responses; CSRF is about preventing
  cookie-auth state changes from other sites. They solve different problems.
- Deployment should keep routes clean (`/api/v1/`, `/app/`, `/admin/`) and avoid
  accidental path conflicts.
- Strong API contracts + tests prevent frontend/backend drift.

---

Next chapter: **Part IX — 36. Production Readiness Checklist**  
We’ll turn your project into a deployable system: hardened settings, static/media
strategy, database backups/migrations plan, CI/CD, and operational runbooks.